# Work in progress, don't review yet

## Infection frequency flow

Let's return to the world of infectious diseases modelling and consider an SIR model,
like the one we introduced in the first notebook (but even simpler, without the death flow).

In [ ]:
def get_sir_freq_model(
    parameters: Dict,
) -> CompartmentalModel:
    """
    Generate an instance of an SIR model with just one fixed transition rate,
    and a frequency-dependent infection process.
    
    Args:
        parameters: The parameter values to be used in running the model
    Returns:
        The summer model object
    """
    
    # Define the 
    compartments = (
        "susceptible",
        "infectious",
        "recovered",
    )
    infectious_compartment = [
        "infectious",
    ]
    analysis_times = (
        parameters["start_time"], 
        parameters["end_time"]
    )

    model = CompartmentalModel(
        times=analysis_times,
        compartments=compartments,
        infectious_compartments=infectious_compartment,
    )
    
    # Assign infectious seed (summer will crash if seed > population)
    pop = parameters["population"]
    seed = parameters["seed"]
    suscept_pop = pop - seed
    
    model.set_initial_population(
        distribution={
            "susceptible": suscept_pop, 
            "infectious": seed}
    )
    
    # Add the frequency-dependent transmission flow
    model.add_infection_frequency_flow(
        name="infection", 
        contact_rate=param("contact_rate"), 
        source="susceptible", 
        dest="infectious",
    )
    
    # Add a constant recovery flow (like the transition flow introduced above)
    model.add_transition_flow(
        name="recovery",
        fractional_rate=param("recovery_rate"), 
        source="infectious", 
        dest="recovered",
    )
    
    return model

### Frequency-dependent transmission
This flow can be used to model infections using frequency-dependent disease transmission.

In simple models like these (without adjustments of infectious status or other stratification features), 
the frequency-dependent infection flow rate (the number of people infected per time unit) can be thought of as:

```python
# contact_rate: Rate at which effective contact occurs
# num_source: Number of people in the (susceptible) source compartment
# num_infectious: Number of in the infectious compartment(s)
# num_pop: Total number of people in the population
force_of_infection = contact_rate * num_infectious / num_pop
flow_rate = force_of_infection * num_source
```

First note that once we have calculated the force of infection,
we multiply this by the size of the source compartment
(which we will often refer to as the susceptible population).
So the force of infection can be thought of in a similar way to the parameter we apply in a transition flow.
That is, the _per capita_ rate of transition from the source to the destination compartment.

In order to calculate the force of infection, we have a parameter (`contact_rate`)
and the proportion (prevalence) of infectious persons in the population
(represented by the last two terms).

Substituting the force of infection formula into the formula for the flow rate and reordering, we can see that:
```python
flow_rate = contact_rate * num_infectious * num_source / num_pop
```
Here the last two terms of the equation together represent the proportion of the total population that is susceptible.
Given that only contacts with susceptible people will result in transmission,
the remaining proportion will be "wasted" from the perspective of the pathogen.

Therefore, this formula can be thought of as the number of infectious persons in the system,
multiplied by the proportion of their contacts that are with susceptible persons,
multiplied by some parameter.
Given this understanding, the parameter can be thought of as the per unit time rate at which
an infectious person comes into contact with other people in the population in such a way as it would result in transmission if that person was susceptible.
This quantity is often referred to as the "effective contact rate",
because it is the daily number of people that a member of this population will come into contact with other members of the population in such a way as it would result in transmission if the contact was between an infectious and a susceptible person.

## Infection density flow

This flow can be used to model infections using density-dependent disease transmission (as opposed to frequency dependent).

In unstratified models, the density-dependent infection flow rate (people infected per time unit) is calculated as:

```python
# contact_rate: Rate at which effective contact happens between two individuals, i.e. contact that would result in transmission were it to occur between a susceptible and an infectious person
# num_source: Number of people in the (susceptible) source compartment
# num_infectious: Number of people infectious
force_of_infection = contact_rate * num_infectious
flow_rate = force_of_infection * num_source
```

In [ ]:
def get_sir_dens_model(
    parameters: Dict,
) -> CompartmentalModel:
    """
    Generate an instance of an SIR model with just one fixed transition rate,
    and a density-dependent infection process.
    
    Args:
        parameters: The parameter values to be used in running the model
    Returns:
        The summer model object
    """
    
    # Define the 
    compartments = (
        "susceptible",
        "infectious",
        "recovered",
    )
    infectious_compartment = [
        "infectious",
    ]
    analysis_times = (
        parameters["start_time"], 
        parameters["end_time"]
    )

    model = CompartmentalModel(
        times=analysis_times,
        compartments=compartments,
        infectious_compartments=infectious_compartment,
    )
    
    model.set_initial_population(
        distribution={
            "susceptible": parameters["population"] - parameters["seed"], 
            "infectious": parameters["seed"]}
    )
    
    # Density-dependent transmission flow, instead of frequency-dependent
    model.add_infection_density_flow(
        name="infection", 
        contact_rate=param("contact_rate"), 
        source="susceptible", 
        dest="infectious",
    )
    
    model.add_transition_flow(
        name="recovery",
        fractional_rate=param("recovery_rate"), 
        source="infectious", 
        dest="recovered",
    )
    
    return model

Because density-dependent transmission does not involve division by the population size,
we need a contact rate that is smaller than what we needed previously by this factor
in order to recover the same dynamics.

In [ ]:
sir_dens_model = get_sir_dens_model(sir_params)

sir_dens_params = copy.copy(sir_params)
sir_dens_params.update({"contact_rate": 0.001})

sir_dens_model.run(parameters=sir_dens_params)
compartment_values = sir_dens_model.get_outputs_df()
compartment_values.plot()